In [3]:
import numpy as np
import pandas as pd
from imdb import Cinemagoer
from tqdm import tqdm
import threading
import pyspark
from pyspark.sql import SQLContext, SparkSession
import sys
import warnings
warnings.simplefilter(action="ignore")

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"

# get a spark context
import pyspark.pandas as ps

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("PySparkPostgresApp") \
    .config("spark.jars", "/project/postgresql-42.3.2.jar") \
    .getOrCreate()

spark



postgres_uri = "jdbc:postgresql://database-2.czd0w3yxirax.eu-west-2.rds.amazonaws.com:5432/imdb_database"
user = "postgres"
password = "qwerty123"

def dbToSpark(dbtable):
    
    try:
        df = spark.read \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", dbtable) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .load()
        return df

    except Exception as ex:
        return ex


def sparkToDB(df_spark,dbtable):
    try:
        df_spark.write \
            .mode("append") \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", dbtable) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .save()
    except Exception as ex:
        return ex
    
def fetchForeignKey(df,dbtable,mappingColumnOutset,mappingColumnTarget,mappedValue,newcol):
    
    df_curr = dbToSpark(dbtable)
    df_curr_pd = df_curr.toPandas()
    df[newcol] = df[mappingColumnOutset].map(df_curr_pd.set_index([mappingColumnTarget])[mappedValue])
    df.drop(columns=[mappingColumnOutset], inplace=True)
    return df

In [4]:
def writeToDB(movies,moviePersons,movieGenres):
    moviesDF = pd.DataFrame(movies)
    moviePersonsDF = pd.concat(moviePersons)
    movieGenresDF = pd.concat(movieGenres)
    
    moviesDF_spark = spark.createDataFrame(moviesDF)
    sparkToDB(moviesDF_spark,"imdb_schema.movie_primary")
    
    genresPresent_df_spark = dbToSpark("imdb_schema.genres")
    genresPresent_df = genresPresent_df_spark.toPandas()
    newGenres = list(set(movieGenresDF['genre_name'])-set(genresPresent_df['genre_name']))
    if len(newGenres)>0:
        newGenresDF = pd.DataFrame()
        newGenresDF['genre_name'] = newGenres
        newGenresDF_spark = spark.createDataFrame(newGenresDF)
        sparkToDB(newGenresDF_spark,"imdb_schema.genres")
    
    personPresent_df_spark = dbToSpark("imdb_schema.person_data")
    personPresent_df = personPresent_df_spark.toPandas()
    newPersons = list(set(moviePersonsDF['person_name'])-set(personPresent_df['person_name']))
    if len(newPersons)>0:
        newPersonsDF = pd.DataFrame()
        newPersonsDF['person_name'] = newPersons
        newPersonsDF_spark = spark.createDataFrame(newPersonsDF)
        sparkToDB(newPersonsDF_spark,"imdb_schema.person_data")
    
    rolesPresent_df_spark = dbToSpark("imdb_schema.role_data")
    rolesPresent_df = rolesPresent_df_spark.toPandas()
    newRoles = list(set(moviePersonsDF['role_name'])-set(rolesPresent_df['role_name']))
    if len(newRoles)>0:
        newRolesDF = pd.DataFrame()
        newRolesDF['role_name'] = newRoles
        newRolesDF_spark = spark.createDataFrame(newRolesDF)
        sparkToDB(newRolesDF_spark,"imdb_schema.role_data")
    
    movieGenresDF = fetchForeignKey(movieGenresDF,"imdb_schema.genres","genre_name",
                                    "genre_name","genre_id","genre_id")
    movieGenresDF_spark = spark.createDataFrame(movieGenresDF)
    sparkToDB(movieGenresDF_spark,"imdb_schema.movie_genres")
    
    moviePersonsDF =fetchForeignKey(moviePersonsDF,"imdb_schema.person_data","person_name",
                                    "person_name","person_id","person_id")
    moviePersonsDF =fetchForeignKey(moviePersonsDF,"imdb_schema.role_data","role_name",
                                "role_name","role_id","role_id")
    moviePersonsDF_spark = spark.createDataFrame(moviePersonsDF)
    sparkToDB(moviePersonsDF_spark,"imdb_schema.movie_person")



In [14]:
#global movies, moviePersons, movieGenres
movies = []#[None]*100
moviePersons = []
movieGenres = []
rolesList = ['cast','director','writer']

In [15]:
ia = Cinemagoer()
def fetchData(x):
    try:
        movieDict = {}
        moviePersonDF = pd.DataFrame(columns = ['movie_id','person_name','role_name'])
        genresDF = pd.DataFrame()
        movieID = "0000000" + str(x)
        movieID = movieID[-7:]
        movie = ia.get_movie(movieID)
        movieDict['movie_id'] = movieID
        movieDetails = {'plot_outline':'plot outline','movie_name':'title',
                        'movie_runtime':'runtimes','rating':'rating'}
        for key,value in movieDetails.items():
            try:
                movieDict[key] = movie.get(value)
            except:
                movieDict[key] = ''
            if type(movieDict[key]) == list:
                movieDict[key] = movieDict[key][0]
        if (movieDict['plot_outline']=='') or (movieDict['plot_outline']==None):
            return ''
        #print(movieDict)
        #movies[x-1]=movieDict
        infoAvailable = set(movie.infoset2keys['main'])
        rolesAvaliable = list(set(rolesList).intersection(infoAvailable))
        for role in rolesAvaliable:
            roleDF = pd.DataFrame()
            persons = movie.get(role)
            persons = [str(person) for person in persons]
            try:
                if role == 'cast':
                    persons = persons[:5]
            except:
                pass
            roleDF['movie_id'] = [movieID]*len(persons)
            roleDF['role_name'] = [role]*len(persons)
            roleDF['person_name'] = persons
            moviePersonDF = moviePersonDF.append(roleDF)
        genres = movie.get('genres')
        genresDF['movie_id'] = [movieID]*len(genres)
        genresDF['genre_name'] = genres
        movies.append(movieDict)
        moviePersons.append(moviePersonDF)
        movieGenres.append(genresDF)
        return movieDict,moviePersonDF,genresDF
    except:
        return ''
#     if x%1000==0:
#         writeToDB(movies,moviePersons,movieGenres)
#         movies = []
#         moviePersons = []
#         movieGenres = []

In [44]:
startList = list(range(80001,500000,100))
startList[:5]
endList =  list(range(80100,500100,100))
endList[:5]

[80100, 80200, 80300, 80400, 80500]

In [45]:
pool_size = 16
from multiprocessing.pool import ThreadPool
from timeit import default_timer as timer
import time
pool = ThreadPool(processes=pool_size)
start = timer()
for start,end in tqdm(zip(startList,endList)):
    try:
        for x in range(start,end+1):
            #fetchData(x)
            results = pool.map_async(fetchData, (x,))
        results.get()
        writeToDB(movies,moviePersons,movieGenres)
        movies = []
        moviePersons = []
        movieGenres = []
        time.sleep(3)
    except HTTPError:
        time.sleep(60)
        
pool.close()
pool.join()
end = timer()
    
print("Total time for code to execute:", end - start)
#futures = [executor.submit(try_my_operation, item) for item in items]
#concurrent.futures.wait(futures)

0it [00:00, ?it/s]2022-04-15 16:36:37,941 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080001/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/p

2022-04-15 16:36:37,949 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080009/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/re

CRITICAL:imdbpy:IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080002/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/pytho

2022-04-15 16:36:37,952 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080011/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/re

CRITICAL:imdbpy:IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080010/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/pytho

...

CRITICAL:imdbpy:caught an exception retrieving or parsing "main" info set for mopID "0080098" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 503: Ser

0it [00:00, ?it/s]
2022-04-15 16:36:38,671 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080073/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/

NameError: name 'HTTPError' is not defined

2022-04-15 16:36:38,680 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/__init__.py:833: caught an exception retrieving or parsing "main" info set for mopID "0080073" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 649, in http_error_def

CRITICAL:imdbpy:caught an exception retrieving or parsing "main" info set for mopID "0080088" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 503: Ser

In [23]:
#writeToDB(movies,moviePersons,movieGenres)
#total_successes.get()

In [24]:
ia = Cinemagoer()
movie = ia.get_movie('0094226')#, info=['taglines', 'plot'])
movie.infoset2keys

{'main': ['localized title',
  'cast',
  'genres',
  'runtimes',
  'countries',
  'country codes',
  'language codes',
  'color info',
  'aspect ratio',
  'sound mix',
  'box office',
  'certificates',
  'original air date',
  'rating',
  'votes',
  'cover url',
  'imdbID',
  'plot outline',
  'languages',
  'title',
  'year',
  'kind',
  'original title',
  'director',
  'writer',
  'producer',
  'composer',
  'cinematographer',
  'editor',
  'editorial department',
  'casting director',
  'art direction',
  'set decoration',
  'costume designer',
  'make up',
  'production manager',
  'assistant director',
  'art department',
  'sound crew',
  'special effects',
  'visual effects',
  'stunt performer',
  'camera and electrical department',
  'casting department',
  'costume department',
  'location management',
  'music department',
  'script department',
  'transportation department',
  'miscellaneous crew',
  'akas',
  'production companies',
  'distributors',
  'special effects co

In [18]:
movieGenres

[  movie_id   genre_name
 0  0000001  Documentary
 1  0000001        Short,
   movie_id genre_name
 0  0000006      Short,
   movie_id genre_name
 0  0000005      Short
 1  0000005     Comedy,
   movie_id genre_name
 0  0000002  Animation
 1  0000002      Short,
   movie_id genre_name
 0  0000007      Short
 1  0000007      Sport,
   movie_id genre_name
 0  0000003  Animation
 1  0000003     Comedy
 2  0000003      Short
 3  0000003    Romance,
   movie_id   genre_name
 0  0000008  Documentary
 1  0000008        Short,
   movie_id genre_name
 0  0000004  Animation
 1  0000004      Short,
   movie_id genre_name
 0  0000009      Short
 1  0000009    Romance,
   movie_id   genre_name
 0  0000010  Documentary
 1  0000010        Short,
   movie_id   genre_name
 0  0000012  Documentary
 1  0000012        Short,
   movie_id   genre_name
 0  0000011  Documentary
 1  0000011        Short,
   movie_id genre_name
 0  0000015  Animation
 1  0000015      Short,
   movie_id   genre_name
 0  0000017 

In [ ]:
movieDF = pd.DataFrame(movies)

In [ ]:
movieDF.head()

In [ ]:
100%51

In [ ]:
pd.concat(moviePersons)

In [ ]:
movies
movieDF = pd.DataFrame(movies)
#movieDF['movie_id'] = movieDF['movie_id'].apply(lambda x:str(x))
movieDF_spark = spark.createDataFrame(movieDF)


In [ ]:
movieDF.dtypes

In [ ]:
movieDF_spark.write \
            .mode("append") \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", "imdb_schema.movie_primary") \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .save()

In [13]:
x = []
def foo():
    x.append(1)
    
foo()
x

[1]

In [14]:
foo()
x

[1, 1]

In [9]:
pd.DataFrame(movies)

,movie_id,movie_name,movie_runtime,rating,plot_outline
0,0000007,Corbett and Courtney Before the Kinetograph,1,5.5,James J. Corbett and Peter Courtney meet in a ...
1,0000001,Carmencita,1,5.7,"Presumably, the first woman ever to appear in ..."
2,0000008,Edison Kinetoscopic Record of a Sneeze,1,5.4,A man (Edison's assistant) takes a pinch of sn...
3,0000003,Poor Pierrot,4,6.5,None
4,0000005,Blacksmith Scene,1,6.2,A stationary camera looks at a large anvil wit...
...,...,...,...,...,...
994,0000995,One Touch of Nature,10,7.3,Policeman John Murray is the proud father of a...
995,0000997,The Open Gate,11,6.7,"Hetty is engaged to George, but after her sist..."
996,0000998,Othello,12,5.6,None
997,0000999,Paul Wang's Destiny,None,5.4,None


In [21]:
print(moviePersons)

[  movie_id    person_name role_name
0  0000003  Émile Reynaud  director,   movie_id           person_name role_name
0  0000001  William K.L. Dickson  director
0  0000001            Carmencita      cast,   movie_id           person_name role_name
0  0000006  William K.L. Dickson  director,   movie_id           person_name role_name
0  0000007  William K.L. Dickson  director
1  0000007         William Heise  director
0  0000007      James J. Corbett      cast
1  0000007        Peter Courtney      cast,   movie_id    person_name role_name
0  0000004  Émile Reynaud  director,   movie_id           person_name role_name
0  0000005  William K.L. Dickson  director
0  0000005        Charles Kayser      cast
1  0000005              John Ott      cast,   movie_id    person_name role_name
0  0000002  Émile Reynaud  director,   movie_id           person_name role_name
0  0000008  William K.L. Dickson  director
0  0000008              Fred Ott      cast,   movie_id        person_name role_name
0  0

In [51]:
moviesDF_spark = dbToSpark("imdb_schema.movie_primary")
#moviesDF = moviesDF_spark.toPandas()

movieGenresDF_spark = dbToSpark("imdb_schema.movie_genres")
#movieGenresDF = movieGenresDF_spark.toPandas()

genresDF_spark= dbToSpark("imdb_schema.genres")
#genresDF = genresDF_spark.toPandas()

roles_data_df_spark = dbToSpark("imdb_schema.role_data")
#roles_data_df = roles_data_df_spark.toPandas()

moviePersonsDF_spark = dbToSpark("imdb_schema.movie_person")
#moviePersonsDF = moviePersonsDF_spark.toPandas()

person_data_df_spark = dbToSpark("imdb_schema.person_data")
#person_data_df = person_data_df_spark.toPandas()

# moviesDF['genre_id'] = moviesDF['movie_id'].map(movieGenresDF.set_index(['movie_id'])['genre_id'])
# moviesDF['genre_name'] = moviesDF['genre_id'].map(genresDF.set_index(['genre_id'])['genre_name'])



In [53]:
movieGenresDF = movieGenresDF_spark.toPandas()
genresDF = genresDF_spark.toPandas()
roles_data_df = roles_data_df_spark.toPandas()
moviePersonsDF = moviePersonsDF_spark.toPandas()

In [69]:
genresDF.shape

(28, 2)

In [59]:
person_data_df_spark = dbToSpark("imdb_schema.person_data")
person_data_df = person_data_df_spark.toPandas()

In [61]:
moviesDF = moviesDF_spark.toPandas()

Py4JJavaError: An error occurred while calling o157441.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8172.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8172.0 (TID 17976) (cube-19972c0e-440b-44b4-9df4-d9f5c4fc2d36-b488f9bbf-m9tzm executor driver): org.postgresql.util.PSQLException: Bad value for type BigDecimal : NaN
	at org.postgresql.jdbc.PgResultSet.toBigDecimal(PgResultSet.java:3212)
	at org.postgresql.jdbc.PgResultSet.toBigDecimal(PgResultSet.java:3221)
	at org.postgresql.jdbc.PgResultSet.getNumeric(PgResultSet.java:2649)
	at org.postgresql.jdbc.PgResultSet.getBigDecimal(PgResultSet.java:2601)
	at org.postgresql.jdbc.PgResultSet.getBigDecimal(PgResultSet.java:444)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$makeGetter$3(JdbcUtils.scala:418)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$makeGetter$3$adapted(JdbcUtils.scala:416)
	at org.apache.spark.sql.execu

In [64]:
from postgres_wrapper import PGWrapper
config = {
    "database": "imdb_database",
    "user": "postgres",
    "password": "qwerty123",
    "host": "database-2.czd0w3yxirax.eu-west-2.rds.amazonaws.com",
    "port": 5432,
}

db_con = PGWrapper(config)
db_con.connect()
results = db_con.query_as_list_of_dicts("SELECT * FROM imdb_schema.movie_primary")

In [66]:
moviesDF=pd.DataFrame(results)

In [73]:
moviesDF.shape

(30187, 6)

In [70]:
def fetchGenres(movieID):
    movieGenres = movieGenresDF[movieGenresDF['movie_id']==movieID]
    movieGenres['genre_name']= movieGenres['genre_id'].map(genresDF.set_index(['genre_id'])['genre_name'])
    return movieGenres['genre_name'].tolist()

In [71]:
moviesDF['genres'] = moviesDF['movie_id'].apply(lambda x: fetchGenres(x))